In [1]:
# GridWorld v1.0

import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import seaborn as sns
import random
import os

class GridWorld(object):
    def __init__(self, m = 20, n = 20, foodNumber = 8, agentsNumber = 4):
        self.m = m
        self.n = n
        self.foodNumber = foodNumber
        self.agentsNumber = agentsNumber
        
        self.grid = np.zeros((m, n))
        self.foodLocation = np.random.randint(20, size=(foodNumber, 2))
        self.setFoodsLoc(self.foodLocation)
        # self.setFoods(foodNumber)
        self.agentAttribute = {}
        self.agentAttribute = self.setAgents(agentsNumber)
        self.possibleActions = ['up', 'down', 'left', 'right', 'eat', 'attack', 'attacked']


    def setFoodsLoc(self, foodLocation):
        for i in range(len(foodLocation)):
            xPos = foodLocation[i][0]
            yPos = foodLocation[i][1]
            if self.grid[xPos][yPos] == 0:
                self.grid[xPos][yPos] = 10
            else:
                i -=1

    def setFoods(self, foodNumber):
        # np.random.seed(2021)
        self.foodNumber = foodNumber
        for i in range(self.foodNumber):
            xPos = np.random.randint(self.m)
            yPos = np.random.randint(self.n)
            if self.grid[xPos][yPos] == 0:
                self.grid[xPos][yPos] = 10
            else:
                i -=1

    def setAgents(self, agentsNumber):
        self.agentsNumber = agentsNumber
        for i in range(agentsNumber):
            xPos = np.random.randint(self.m)
            yPos = np.random.randint(self.n)
            if self.grid[xPos][yPos] == 0:
                self.grid[xPos][yPos] = 1 + i
                self.agentAttribute[i]={'xPos': xPos, 'yPos': yPos, 'reward': 0}
            else:
                i -=1
        return self.agentAttribute

    def step(self , action, agentID):
        # 'attack':2, 'attacked':-4
        reward_dict = {'up':-1, 'down':-1, 'left':-1, 'right':-1, 'eat':1, 'attack':2, 'attacked':-4, 'wrong attack':-1}
        
        if action == "up" and self.agentAttribute[agentID]['yPos'] - 1 > -1 and self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos'] - 1] == 0:
            self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos']] = 0
            self.agentAttribute[agentID]['yPos'] -= 1
            self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos']] = agentID + 1
            # print("agent ", agn," goes up")
            return self.agentAttribute, reward_dict['up'], self.status(self.agentAttribute) , None
        elif action == "down" and self.agentAttribute[agentID]['yPos'] + 1 < 20 and self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos'] + 1] == 0:
            self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos']] = 0
            self.agentAttribute[agentID]['yPos'] += 1
            self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos']] = agentID + 1
            # print("agent ", agn," goes down")
            return self.agentAttribute, reward_dict['down'], self.status(self.agentAttribute) , None
        elif action == "right" and self.agentAttribute[agentID]['xPos'] + 1 < 20 and self.grid[self.agentAttribute[agentID]['xPos'] + 1][self.agentAttribute[agentID]['yPos']] == 0:
            self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos']] = 0
            self.agentAttribute[agentID]['xPos'] += 1
            self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos']] = agentID + 1
            # print("agent ", agn," goes right")
            return self.agentAttribute, reward_dict['right'], self.status(self.agentAttribute) , None
        elif action == "left" and self.agentAttribute[agentID]['xPos'] - 1 > -1 and self.grid[self.agentAttribute[agentID]['xPos'] - 1][self.agentAttribute[agentID]['yPos']] == 0:
            self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos']] = 0
            self.agentAttribute[agentID]['xPos'] -= 1
            self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos']] = agentID + 1
            # print("agent ", agn," goes left")
            return self.agentAttribute, reward_dict['left'], self.status(self.agentAttribute) , None
        elif action == "eat":
            # print("agent ", agn," in eat action", self.agentAttribute[agentID], "\t Grid status", self.grid[self.agentAttribute[agentID]['xPos'] - 1][self.agentAttribute[agentID]['yPos']])
            if self.agentAttribute[agentID]['xPos'] - 1 > -1 and self.grid[self.agentAttribute[agentID]['xPos'] - 1][self.agentAttribute[agentID]['yPos']] == 10:
                # print("agent ", agn," in eated in # 1 !!!!", self.agentAttribute[agentID])
                return self.agentAttribute, reward_dict['eat'], self.status(self.agentAttribute) , None
            elif self.agentAttribute[agentID]['xPos'] + 1 < 20 and self.grid[self.agentAttribute[agentID]['xPos'] + 1][self.agentAttribute[agentID]['yPos']] == 10:
                # print("agent ", agn," in eated in # 2 !!!!", self.agentAttribute[agentID])
                return self.agentAttribute, reward_dict['eat'], self.status(self.agentAttribute) , None
            elif self.agentAttribute[agentID]['yPos'] - 1 > -1 and self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos'] - 1] == 10:
                # print("agent ", agn," in eated in # 3 !!!!", self.agentAttribute[agentID])
                return self.agentAttribute, reward_dict['eat'], self.status(self.agentAttribute) , None
            elif self.agentAttribute[agentID]['yPos'] + 1 < 20 and self.grid[self.agentAttribute[agentID]['xPos']][self.agentAttribute[agentID]['yPos'] + 1] == 10:
                # print("agent ", agn," in eated in # 4 !!!!", self.agentAttribute[agentID])
                return self.agentAttribute, reward_dict['eat'], self.status(self.agentAttribute) , None
            elif self.agentAttribute[agentID]['xPos'] - 1 > -1 and self.agentAttribute[agentID]['yPos'] - 1 > -1 and self.grid[self.agentAttribute[agentID]['xPos'] - 1 ][self.agentAttribute[agentID]['yPos'] - 1] == 10:
                # print("agent ", agn," in eated in # 5 !!!!", self.agentAttribute[agentID])
                return self.agentAttribute, reward_dict['eat'], self.status(self.agentAttribute) , None
            elif self.agentAttribute[agentID]['xPos'] - 1 > -1 and self.agentAttribute[agentID]['yPos'] + 1 < 20 and self.grid[self.agentAttribute[agentID]['xPos'] - 1 ][self.agentAttribute[agentID]['yPos'] + 1] == 10:
                # print("agent ", agn," in eated in # 6 !!!!", self.agentAttribute[agentID])
                return self.agentAttribute, reward_dict['eat'], self.status(self.agentAttribute) , None
            elif self.agentAttribute[agentID]['xPos'] + 1 < 20 and self.agentAttribute[agentID]['yPos'] - 1 > -1 and self.grid[self.agentAttribute[agentID]['xPos'] + 1 ][self.agentAttribute[agentID]['yPos'] - 1] == 10:
                # print("agent ", agn," in eated in # 7 !!!!", self.agentAttribute[agentID])
                return self.agentAttribute, reward_dict['eat'], self.status(self.agentAttribute) , None
            elif self.agentAttribute[agentID]['xPos'] + 1 < 20 and self.agentAttribute[agentID]['yPos'] + 1 < 20 and self.grid[self.agentAttribute[agentID]['xPos'] + 1 ][self.agentAttribute[agentID]['yPos'] + 1] == 10:
                # print("agent ", agn," in eated in # 8 !!!!", self.agentAttribute[agentID])
                return self.agentAttribute, reward_dict['eat'], self.status(self.agentAttribute) , None
            else:
                # print("agent ", agn," in take another action !!!!")
                action = env.actionSpaceSample()
                return self.step(action, agentID)

        elif action == "attack":
            for i in range(len(self.agentAttribute)):
                if i == agentID:
                    continue
                if -2 < self.agentAttribute[agentID]['xPos'] - self.agentAttribute[i]['xPos'] and self.agentAttribute[agentID]['xPos'] - self.agentAttribute[i]['xPos'] < 2:
                    if -2 < self.agentAttribute[agentID]['yPos'] - self.agentAttribute[i]['yPos'] and self.agentAttribute[agentID]['yPos'] - self.agentAttribute[i]['yPos'] < 2:
                        # self.agentAttribute[i]['reward'] += reward_dict['attacked']
                        # print("agent: ", agn," Attacked agent: ",i , "!!!!", self.agentAttribute[agentID], self.agentAttribute[i])
                        return self.agentAttribute, reward_dict['attack'], self.status(self.agentAttribute) , {'action':'Attack','agentID': i,'reward': reward_dict['attacked']}
            return self.agentAttribute, reward_dict['wrong attack'], self.status(self.agentAttribute) , None
        else:
            action = env.actionSpaceSample()
            return self.step(action, agentID)

    def status(self, agentAttribute):
        self.agentAttribute = agentAttribute
        if len(self.agentAttribute) < 2:
            return True
        else:
            return False

    def reset(self):
        self.grid = np.zeros((self.m, self.n))
        self.setFoodsLoc(self.foodLocation)
        # self.setFoods(self.foodNumber)
        self.agentAttribute = self.setAgents(self.agentsNumber)
        return self.agentAttribute

    def render(self):
        print('------------------------------------------')
        for row in self.grid:
            for col in row:
                if col == 0:
                    print('-', end='  ')
                elif col == 10:
                    print('F', end='  ')
                elif col == 1:
                    print('X1', end='  ')
                elif col == 2:
                    print('X2', end='  ')
                elif col == 3:
                    print('X3', end='  ')
                elif col == 4:
                    print('X4', end='  ')
            print('\n')
        print('------------------------------------------')

    def actionSpaceSample(self):
        return np.random.choice(self.possibleActions)

def maxAction(Q, agentID, r, c, actions):
    values = np.array([Q[agentID, r, c, a] for a in actions])
    action = np.argmax(values)
    return actions[action]

def debug(Q_last, Q):
    for agn in range(agentsNumber):
        for r in range(row):
            for col in range(column):
                if Q[agn, r, col, 'attack'] != Q_last[agn, r, col, 'attack']:
                    print("agentID:" + str(agn) + "r:" + str(r) + "col:" + str(col) +" attack: " + str(Q[agn, r, col, 'attack']))

def log(i, episodeReward, agentsNumber, row, column, Q):
    if i % 100 == 0:
        print("Starting episode: ", i)
        print("episodeReward: ", episodeReward)

    if i % 1000 == 0:
        for agn in range(agentsNumber):
            tempEat=[[0 for i in range(row)] for j in range(column)]
            tempAttack=[[0 for i in range(row)] for j in range(column)]
            for r in range(row):
                for col in range(column):
                    tempEat[r][col] = round(Q[agn, r, col, 'eat'], 1)
                    tempAttack[r][col] = round(Q[agn, r, col, 'attack'], 1)
            plt.figure(figsize=(16, 10))
            sns.heatmap(tempEat, annot=True)
            plt.savefig("./log/episode_" + str(i) + "_agent_Eat_Behav_" + str(agn) + ".png")
            plt.close()
            plt.figure(figsize=(16, 10))
            sns.heatmap(tempAttack, annot=True)
            plt.savefig("./log/episode_" + str(i) + "_agent_Attack_Behav_" + str(agn) + ".png")
            plt.close()

        for agn in range(agentsNumber):
            with open("./log/Q_episode" + str(i) + "AgentID_" + str(agn) + ".txt", 'w') as f:
                for r in range(row):
                    f.write("\n")
                    for col in range(column):
                        for action in env.possibleActions:
                            f.write("agentID:" + str(agn) + "r:" + str(r) + "col:" + str(col) +"action:" + str(action) + "," + str(Q[agn, r, col, action]) + "//  ")
        
def eval(env, Q):
    done = False
    episodeReward = [0 for i in range(agentsNumber)]
    agentAttr = env.reset()
    doneCounter = 0
    while not done:
        doneCounter += 1
        for agn in agentAttr:
            rand = np.random.random()
            if rand < (1-EPS):
                action = maxAction(Q, agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], env.possibleActions)
                # print("action from Q function")
            else:
                action = env.actionSpaceSample()
                # print("action from random sample")

            # print("agent: ", agn, "take action: ", action)
            agentAttr_, reward, done, info = env.step(action, agn)
            agentAttr = agentAttr_
            env.render()
            time.sleep(1)
            if doneCounter > 30:
                done = True


In [ ]:

if __name__ == '__main__':
    # map magic squares to their connecting square
    row = 20
    column = 20
    foodNumber = 8
    agentsNumber = 4
    episodeReward = [0 for i in range(agentsNumber)]
    agentlist = [i for i in range(agentsNumber)]
    env = GridWorld(row, column, foodNumber, agentsNumber)
    # model hyperparameters
    ALPHA = 0.01
    GAMMA = 1.0
    EPS = 1.0
    if not os.path.exists('./log'):
        os.mkdir('./log')
    Q = {}
    for agn in range(agentsNumber):
        for r in range(row):
            for col in range(column):
                for action in env.possibleActions:
                    Q[agn, r, col, action] = 0

    episode = 10000
    totalRewards = [0 for j in range(episode)]
    Q_last = Q
    
    for i in range(episode):
        # print([[env.agentAttribute[i]['xPos'], env.agentAttribute[i]['yPos']] for i in range(agentsNumber)])
        # print(env.foodLocation)
        log(i, episodeReward, agentsNumber, row, column, Q)
        done = False
        episodeReward = [0 for i in range(agentsNumber)]
        agentAttr = env.reset()
        doneCounter = 0
        while not done:
            doneCounter += 1
            # print('episodeReward ', episodeReward)
            random.shuffle(agentlist)
            for agn in agentlist:
                rand = np.random.random()
                if rand < (1-EPS):
                    action = maxAction(Q, agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], env.possibleActions) 
                else:
                    action = env.actionSpaceSample()
                
                # agentAttr_, reward, done, info = env.step(action, agn)
                # episodeReward[agn] += reward
                # action_ = maxAction(Q, agn, agentAttr_[agn]['xPos'], agentAttr_[agn]['yPos'], env.possibleActions)
                # Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action] = Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action] + ALPHA*(reward + GAMMA*Q[agn, agentAttr_[agn]['xPos'], agentAttr[agn]['yPos'], action_] - Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action])
                # agentAttr = agentAttr_

                agentAttr_, reward, done, info = env.step(action, agn)
                if isinstance(info, type(None)):
                    episodeReward[agn] += reward
                    action_ = maxAction(Q, agn, agentAttr_[agn]['xPos'], agentAttr_[agn]['yPos'], env.possibleActions)
                    Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action] = Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action] + ALPHA*(reward + GAMMA*Q[agn, agentAttr_[agn]['xPos'], agentAttr[agn]['yPos'], action_] - Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action])
                    agentAttr = agentAttr_
                else:
                    episodeReward[info['agentID']] += info['reward']
                    action_ = maxAction(Q, info['agentID'], agentAttr_[info['agentID']]['xPos'], agentAttr_[info['agentID']]['yPos'], env.possibleActions)
                    Q[info['agentID'], agentAttr[info['agentID']]['xPos'], agentAttr[info['agentID']]['yPos'], action] = Q[info['agentID'], agentAttr[info['agentID']]['xPos'], agentAttr[info['agentID']]['yPos'], action] + ALPHA*(reward + GAMMA*Q[info['agentID'], agentAttr_[info['agentID']]['xPos'], agentAttr[info['agentID']]['yPos'], action_] - Q[info['agentID'], agentAttr[info['agentID']]['xPos'], agentAttr[info['agentID']]['yPos'], action])

                    episodeReward[agn] += reward
                    action_ = maxAction(Q, agn, agentAttr_[agn]['xPos'], agentAttr_[agn]['yPos'], env.possibleActions)
                    Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action] = Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action] + ALPHA*(reward + GAMMA*Q[agn, agentAttr_[agn]['xPos'], agentAttr[agn]['yPos'], action_] - Q[agn, agentAttr[agn]['xPos'], agentAttr[agn]['yPos'], action])
                    
                    agentAttr = agentAttr_

                if doneCounter > 100:
                    done = True

        if EPS - 2 / episode > 0:
            EPS -= 2 / episode
        else:
            EPS = 0

        totalRewards[i] = episodeReward

    TRS = np.asarray(totalRewards)
    for i in range(agentsNumber):
        plt.figure(figsize=(16, 10))
        plt.plot(TRS[:,i])
        plt.savefig("./log/totalRewardAgent_" + str(i) + ".png")
        plt.close()
    
    eval(env, Q)